# Ollama + Mistral 7B LLM RAG (Retrieval Augmented Generation) using LangChain

You can use this to ask LLM anything about you pdf file. Make sure that you have installed Ollama
See installation guide on official documentation -> https://ollama.com/

## Install the necessary packages

In [ ]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"
%pip install --q chromadb
%pip install --q langchain-text-splitters

## Import the necessary modules

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

## Instll necessary models for Ollama

In [ ]:
!ollama pull nomic-embed-text
!ollama pull mistral
!ollama list

## LLM

In [ ]:
llm = ChatOllama(model="mistral")

## Loading file

In [ ]:
file_path = "./data.pdf"
loader = UnstructuredPDFLoader(file_path)
data = loader.load()

# To make sure that the document is loaded
# Uncommend the following line to preview the first page
# data[0].page_content

## Splitting data into small chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

## Vector database

In [ ]:
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

## Defining prompt template for the LLM

In [ ]:
QUERY = PromptTemplate(
    input_variables=["question"],
    template="""
    Your are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relevant documents 
    from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}
    """
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(), llm, prompt=QUERY)



In [ ]:
template = """
Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

## Define the LLM chain

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("what is it about ?")

## Clean vector db collection

In [ ]:
vector_db.delete_collection()